In [43]:
class Category:
    def __init__(self, name):
        self.name = name
        self.ledger = []


    def __str__(self):
        title = f"{self.name:*^30}\n"
        items = ""
        total = 0
        for item in self.ledger:
            item += f"{item['description'][0:23]:23}" + f"{item['amount']:>7.2f}" + '\n'

            total += item['amount']

        output = title + items + "Total: " + str(total)
        return output

    def deposit(self, amount, description=""):
        self.ledger.append({'amount': amount, 'description': description})

    def withdraw(self, amount, description=""):
        if self.check_funds(amount):
            self.ledger.append({'amount': -amount, 'description': description})
            return True
        return False

    def get_balance(self):
        balance = 0
        for item in self.ledger:
            balance += item['amount']
        return balance

    def transfer(self, amount, category):
        if self.check_funds(amount):
            self.withdraw(amount, f"Transfer to {category.name}")
            category.deposit(amount, f"Transfer from {self.name}")
            return True
        return False

    def check_funds(self, amount):
        return amount <= self.get_balance()

    def __str__(self):
        title = f"*************{self.name}*************"
        output = title + "\n"
        for item in self.ledger:
            description = item['description'][:23]
            amount = str(item['amount']).rjust(7)
            output += f"{description}{amount}\n"
        output += f"Total: {self.get_balance():.2f}"
        return output

food = Category('Food')
food.deposit(1000, 'deposit')
food.withdraw(10.15, 'groceries')
food.withdraw(15.89, 'restaurant and more food for dessert')
clothing = Category('Clothing')
food.transfer(50, clothing)
print(food)

rent = Category('Rent')
rent.deposit(120, 'deposit')
rent.withdraw(10.15, 'water')
rent.withdraw(15.89, 'electricity')
food = Category('Food')
rent.transfer(38, food)
print(rent)

car = Category('Car')
car.deposit(3500, 'deposit')
car.withdraw(350.15, 'gas')
car.withdraw(150.89, 'oil')
clothing = Category('Clothing')
food.transfer(560, clothing)
print(car)

*************Food*************
deposit   1000
groceries -10.15
restaurant and more foo -15.89
Transfer to Clothing    -50
Total: 923.96
*************Rent*************
deposit    120
water -10.15
electricity -15.89
Transfer to Food    -38
Total: 55.96
*************Car*************
deposit   3500
gas-350.15
oil-150.89
Total: 2998.96


In [41]:
## Sample

food = Category("Food")
entertainment = Category("Entertainment")
business = Category("Business")
food.deposit(900, "deposit")
entertainment.deposit(900, "deposit")
business.deposit(900, "deposit")
food.withdraw(105.55)
entertainment.withdraw(33.40)
business.withdraw(10.99)
print(create_spend_chart([business, food, entertainment]))

Percentage spent by category
100|          
 90|          
 80|          
 70|    o     
 60|    o     
 50|    o     
 40|    o     
 30|    o     
 20|    o  o  
 10|    o  o  
  0| o  o  o  
    ----------
     B  F  E  
     u  o  n  
     s  o  t  
     i  d  e  
     n     r  
     e     t  
     s     a  
     s     i  
           n  
           m  
           e  
           n  
           t  


In [45]:
class Category:
    categories = []

    def __init__(self, category):
        self.ledger = []
        self.category = category
        self.ledger_print = ""

    def __str__(self):
        return self.create_ledger_print()

    def add_category(self):
        if self not in Category.categories:
            Category.categories.append(self)
        return

    def deposit(self, amount, description=""):
        self.ledger.append({"amount": amount, "description": description})
        self.add_category()
        return

    def withdraw(self, amount, description=""):
        neg_amount = amount * -1
        success = self.check_funds(amount)
        if success:
            self.ledger.append({"amount": neg_amount, "description": description})

        return success

    def transfer(self, amount, target_category):
        success = self.check_funds(amount)
        if success:
            withdraw_description = f"Transfer to {target_category.category}"
            target_category.add_category()
            self.withdraw(amount, withdraw_description)
            deposit_description = f"Transfer from {self.category}"
            target_category.deposit(amount, deposit_description)

        return success

    def check_funds(self, amount):
        success = False
        if self.get_balance() >= amount:
            success = True

        return success

    def get_balance(self):
        return sum(item["amount"] for item in self.ledger)

    def create_ledger_print(self):
        ledger_print = [self.category.center(30, "*"), "\n"]
        for item in self.ledger:
            description = f"{item['description'][:23]:<23}"
            amount = f"{item['amount']:.2f}"[:7]
            ledger_print.append(description + f"{amount:>7}" + "\n")
        ledger_print.append(f"Total: {self.get_balance():.2f}")

        return "".join(ledger_print)


def gather_percentage(categories):
    spendings = {}
    percents = {}
    percents_values = []
    cats_cap = []

    sum_spendings = 0
    for category in categories:
        amount = sum(item["amount"] for item in category.ledger if item["amount"] < 0) * -1
        spendings[category.category] = amount
        sum_spendings += amount

    per_cent = sum_spendings / 100
    for category, total in spendings.items():
        percentage = int(max(0, ((total / per_cent) // 10) * 10))
        percents[category] = percentage

    for item, key in percents.items():
        percents_values.append(key)
        cats_cap.append(str(item).capitalize())

    return percents_values, cats_cap


def construct_seperator(categories):
    seperator = [str(4 * " ")]
    for _ in categories:
        seperator.append(str(3 * "-"))
    seperator.append("-\n")
    target_length = len("".join(seperator))
    return "".join(seperator), (target_length - 1)


def fix_length(target_row_length, row):
    current_row_length = len("".join(row))
    if current_row_length < target_row_length:
        row.append(str((target_row_length - current_row_length) * " "))

    return row


def construct_percent_chart(percents_values, target_row_length):
    percent_chart = []

    for i in range(100, -1, -10):
        row = [f"{i:>3}| "] if i != 0 else [f"{i:>3}|"]
        for item in percents_values:
            row.append(" o " if item >= i else "  ")
        row = fix_length(target_row_length, row)
        row.append("\n")
        percent_chart.append("".join(row))

    return "".join(percent_chart)


def create_cat_chart(cats_cap, target_row_length):
    cat_chart = []
    target_column_length = 0

    for item in cats_cap:
        if len(item) > target_column_length:
            target_column_length = len(item)

    for i in range(0, target_column_length):
        row = []
        for j, item in enumerate(cats_cap):
            if j == 0:
                row.append(str(4 * " "))
            if i < len(item):
                row.append(f" {item[i]} ")
            else:
                row.append(str(3 * " "))
        row = fix_length(target_row_length, row)
        if i < (target_column_length - 1):
            row.append("\n")
        cat_chart.append("".join(row))

    return "".join(cat_chart)


def create_spend_chart(categories=Category.categories):
    spend_chart = ["Percentage spent by category", "\n"]

    percents_values, cats_cap = gather_percentage(categories)
    seperator, target_row_length = construct_seperator(categories)

    spend_chart.append(construct_percent_chart(percents_values, target_row_length))
    spend_chart.append(seperator)
    spend_chart.append(create_cat_chart(cats_cap, target_row_length))

    return "".join(spend_chart)

In [47]:
food = Category("Food")
entertainment = Category("Entertainment")
business = Category("Business")
food.deposit(900, "deposit")
entertainment.deposit(900, "deposit")
business.deposit(900, "deposit")
food.withdraw(78)
entertainment.withdraw(22)
business.withdraw(8)

chart = create_spend_chart([food, entertainment])
print(chart)

Percentage spent by category
100|       
 90|       
 80|       
 70|  o    
 60|  o    
 50|  o    
 40|  o    
 30|  o    
 20|  o  o 
 10|  o  o 
  0| o  o  
    -------
     F  E  
     o  n  
     o  t  
     d  e  
        r  
        t  
        a  
        i  
        n  
        m  
        e  
        n  
        t  


In [49]:
food = Category("Food")
entertainment = Category("Entertainment")
business = Category("Business")
food.deposit(900, "deposit")
entertainment.deposit(900, "deposit")
business.deposit(900, "deposit")
food.withdraw(105.55)
entertainment.withdraw(33.40)
business.withdraw(10.99)
print(create_spend_chart([business, food, entertainment]))

Percentage spent by category
100|          
 90|          
 80|          
 70|    o     
 60|    o     
 50|    o     
 40|    o     
 30|    o     
 20|    o  o  
 10|    o  o  
  0| o  o  o  
    ----------
     B  F  E  
     u  o  n  
     s  o  t  
     i  d  e  
     n     r  
     e     t  
     s     a  
     s     i  
           n  
           m  
           e  
           n  
           t  


In [51]:
# My own output
food = Category("Food")
car = Category("Car")
rent = Category("Rent")
food.deposit(5000, "deposit")
car.deposit(3000, "deposit")
rent.deposit(1000, "deposit")
food.withdraw(150)
car.withdraw(433)
rent.withdraw(200)
print(create_spend_chart([food, car, rent]))

Percentage spent by category
100|          
 90|          
 80|          
 70|          
 60|          
 50|    o     
 40|    o     
 30|    o     
 20|    o  o  
 10|  o  o  o 
  0| o  o  o  
    ----------
     F  C  R  
     o  a  e  
     o  r  n  
     d     t  


In [53]:
class Category:
    def __init__(self, name):
        self.name = name
        self.ledger = []

    def deposit(self, amount, description=""):
        self.ledger.append({"amount": amount, "description": description})

    def withdraw(self, amount, description=""):
        if self.check_funds(amount):
            self.ledger.append({"amount": -amount, "description": description})
            return True
        return False

    def get_balance(self):
        return sum(item["amount"] for item in self.ledger)

    def transfer(self, amount, category):
        if self.check_funds(amount):
            self.withdraw(amount, f"Transfer to {category.name}")
            category.deposit(amount, f"Transfer from {self.name}")
            return True
        return False

    def check_funds(self, amount):
        return amount <= self.get_balance()

    def __str__(self):
        title = f"{self.name:*^30}\n"
        items = ""
        for item in self.ledger:
            description = item["description"][:23]
            amount = f"{item['amount']:.2f}"[:7]
            items += f"{description:<23}{amount:>7}\n"
        total = f"Total: {self.get_balance():.2f}"
        return title + items + total


def create_spend_chart(categories):
    # Calculate total spent and percentages
    total_spent = 0
    category_spent = []

    for category in categories:
        spent = sum(-item["amount"] for item in category.ledger if item["amount"] < 0)
        category_spent.append(spent)
        total_spent += spent

    percentages = [int((spent / total_spent) * 100 // 10) * 10 for spent in category_spent]

    # Build the chart
    chart = "Percentage spent by category\n"
    for i in range(100, -1, -10):
        chart += f"{i:>3}| "
        for percentage in percentages:
            chart += "o  " if percentage >= i else "   "
        chart += "\n"

    # Add the horizontal line
    chart += "    " + "-" * (3 * len(categories) + 1) + "\n"

    # Add the category names vertically
    max_name_length = max(len(category.name) for category in categories)
    names = [category.name.ljust(max_name_length) for category in categories]

    for i in range(max_name_length):
        chart += "     "
        for name in names:
            chart += name[i] + "  "
        chart += "\n"

    return chart.rstrip("\n")

In [55]:

food = Category("Food")
car = Category("Car")
rent = Category("Rent")
food.deposit(5000, "deposit")
car.deposit(3000, "deposit")
rent.deposit(1000, "deposit")
food.withdraw(150)
car.withdraw(433)
rent.withdraw(200)
print(create_spend_chart([food, car, rent]))

Percentage spent by category
100|          
 90|          
 80|          
 70|          
 60|          
 50|    o     
 40|    o     
 30|    o     
 20|    o  o  
 10| o  o  o  
  0| o  o  o  
    ----------
     F  C  R  
     o  a  e  
     o  r  n  
     d     t  
